# Ingest vector Data 

This notebook was added to help test the logic for setting up the vector stores that powers both the natural language semantic search as well as Image semantic search

In [ ]:
from db.vector_store.set_up_vector_store import vectorize_database

In [ ]:
# Fetch the products in the database and create a local vector store that can be used to assist the sql queries
db = vectorize_database()

In [ ]:
docs = db.similarity_search('household appliances', 2)

In [ ]:
[d.metadata for d in docs]

In [ ]:
db.save_local('./ecommerce_index')

# Load Data base From local 

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

In [ ]:
embeddings = OpenAIEmbeddings()

try: 
    new_db = FAISS.load_local("ecommerce_index", embeddings)
    print("successfull")
except:
    print("load failed")

docs = new_db.similarity_search_with_relevance_scores('human hair', k=5)

In [ ]:
[ d.metadata for d, score in docs if score > 0.7]

# Ingest images

In [1]:
from db.vector_store.set_up_vector_store import (vectorize_db_images, 
                                                 get_image_vector_from_file,
                                                 get_similarity)
import pandas as pd

/Users/ikechukwu/Desktop/Hackerton/flask/env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
merged_products, data = vectorize_db_images()

In [ ]:
merged_products.to_pickle('./images_index/product_map.pkl')
pd.DataFrame(data).to_pickle('./images_index/image_embeddings.pkl')

# Load index from local

In [2]:
merged_products = pd.read_pickle('./images_index/product_map.pkl')
embeddings = pd.read_pickle('./images_index/image_embeddings.pkl').T

In [3]:
test_image_path = './images/computer.jpeg'
test_image_embeddings = get_image_vector_from_file(test_image_path)

In [4]:
ids = get_similarity(embeddings, test_image_embeddings, 3)

In [9]:
results = merged_products[merged_products['product_id'].isin(ids)]
[v for k,v in results.T.to_dict().items()]

[{'product_id': 4,
  'name': 'Polystar 32-Inches Smart Curved TV With Netflix',
  'description': 'POLYSTAR 32 INCH SMART CURVED- Upscalling upgrades FHD signal to near 4k quality to deliver powerful details, allowing viewers to enjoy 4K picture quality even at FHD signal. ',
  'price': '₦ 113,900',
  'stock_quantity': 10,
  'category_id': 2,
  'color_id': 1,
  'size_id': 3,
  'product_rating': 2.4,
  'product_image_url': 'https://ng.jumia.is/unsafe/fit-in/500x500/filters:fill(white)/product/65/730317/1.jpg?2455',
  'category': 'Appliance',
  'size': 'L',
  'color': 'Fuscia'},
 {'product_id': 16,
  'name': 'Samsung Galaxy Tab A7 Lite 8.7" TFT 3GB RAM/32GB ROM Android 11 - Gray',
  'description': 'Scanfrost Mist Rechargeable Fan with Remote 16" - SFRF161K is a new product from Scanfrost.  ',
  'price': '₦ 152,889',
  'stock_quantity': 18,
  'category_id': 3,
  'color_id': 50,
  'size_id': 4,
  'product_rating': 4.5,
  'product_image_url': 'https://ng.jumia.is/unsafe/fit-in/500x500/filter